# GO markers of human and mouse populations

**TO RUN THIS NOTEBOOK YOU HAVE TO RUN 4H and 4M NOTEBOOKS FULLY!!!**

## imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import scipy.sparse as spr
import networkx as nx
import gseapy as gp
from matplotlib import pylab

In [ ]:
!pip install cellassign

In [ ]:
# local imports and imports from other notebooks
from cellassign import assign_cats
from fb_functions import make_gene_scoring_with_expr, plot_score_graph, plot_UMAPS_gene, plot_adata_cluster_properties, make_dicts_fraction_mean, plot_dotplot_gene
%store -r dict_colors_human
%store -r seed
%store -r magma
%store -r data_dir
%store -r dict_cats_fb_human

In [ ]:
%store -r plot_params

pylab.rcParams.update(plot_params)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
def retrieve_enrichr(list_genes, org='Human', name='sample', pval=0.05):
    enr = gp.enrichr(gene_list=list_genes,
                 gene_sets=['GO_Biological_Process_2021','GO_Cellular_Component_2021', 'GO_Molecular_Function_2021', 
                            'Reactome 2022', 'WikiPathway_2021_Human', 'KEGG_2021_Human'],
                 organism=org, # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 outdir='test/enrichr/{sample}',
                 # no_plot=True,
                 cutoff=0.5 # test dataset, use lower value from range(0,1)
                )
    
    res = enr.results.sort_values(by='Adjusted P-value')
    res = res[res['Adjusted P-value'] < pval][['Gene_set', 'Term', 'Adjusted P-value', 'Combined Score', 'Genes']]
    res['Population'] = name
    return res

In [ ]:
N = 100

## Creation of list of genes
To do that, we use a mixture of the list of genes obtained in Notebook 4H and 4M, and in the Gene Info table.

In [ ]:
df_GOs = pd.DataFrame(columns=['Population', 'Gene_set', 'Term', 'Adjusted P-value', 'Combined Score', 'Genes'])

In [ ]:
human_A1 = ['AGTR1', 'ANGPTL5', 'ARFGEF3', 'CA12', 'CADM3', 'CD24', 'CD55', 'CD70', 'CDA', 'CGREF1', 'CLEC3B', 'CPVL', 
            'CYBRD1', 'DKK1', 'DPP4', 'EYA3', 'FZD6', 'GALNT15', 'GDF15', 'GLRB', 'LGR5', 'LIMS2', 'MTCL1', 'NPR1', 'RAB32', 'TUBB4A', 
            'ABCB5', 'ABCC9', 'ACKR3', 'ACKR4', 'ADA', 'ADGRD1', 'AEBP1', 'AOX1', 'CHRDL1', 'CPZ', 'CTHRC1', 'CYP4B1', 'DCN', 
            'ENTPD1', 'FAP', 'FBLN1', 'FBLN2', 'GPNMB', 'HPGD', 'ISLR', 'LOX', 'LOXL1', 'MEDAG', 'MGST1', 'MMP27', 'MTSS1', 'NTN1',
            'OMD', 'PCOLCE', 'PI16', 'PIEZO2', 'PODN', 'PRKG2', 'QPCT', 'SCARA5', 'SEMA3B', 'SERPINF1', 'SGCA', 'SGCG', 'SLPI', 'SMOC2', 
            'THBS3', 'TNXB', 'TSPAN8', 'WISP1', 'CCN4', 'WISP2', 'CCN5']

res_A1 = retrieve_enrichr(human_A1, org='Human', name='A1')
df_GOs = pd.concat([df_GOs, res_A1.iloc[:N]])
res_A1

In [ ]:
human_A2 = ['AHRR2', 'AKAP6', 'ANTXR1', 'APELA', 'C1orf98', 'CLEC2A', 'CMKLR1', 'COL13A1', 'COL18A1', 'COL23A1', 'EDN3', 'EFNA5',
            'F13A1', 'F5', 'FGFR2', 'HSPB3', 'KCNQ3', 'LINC00327', 'MAP2', 'MAP7', 'MOXD1', 'MYO1D', 'NCKAP5', 'NTM', 'OSBP2', 'PCSK2', 
            'PHACTR3', 'PREX1', 'PTGS1', 'COX1', 'PTPRE', 'RAMP3', 'RGS2', 'RSPO3', 'SCARF2', 'SHISAL1', 'STC1', 'STC2', 'THBD', 'TMEM52', 'TMTC2', 
            'TNFRSF19', 'TTR', 'APCDD1', 'AXIN2', 'CD9', 'DCD42EP3', 'COL6A1', 'COL6A2', 'COL6A3', 'COL6A5', 'COMP', 'CYP26B1', 'EPHB6', 'GREM2', 'ISM1',
            'KCNK2', 'LEPR', 'LSAMP', 'NKD2', 'NPTX2', 'PHLDA1', 'PTK7', 'RSPO1', 'SMIM3', 'STMN1', 'THSD1', 'THSD4', 'TNC', 'TNFRSF21', 'WNT11']

res_A2 = retrieve_enrichr(human_A2, org='Human', name='A2')
df_GOs = pd.concat([df_GOs, res_A2.iloc[:N]])
res_A2

In [ ]:
human_A3 = ['ANGPTL5', 'ARFGEF3', 'CES1', 'COL14A1', 'CORIN', 'CPZ', 'ELN', 'ENPP1', 'SGC1', 'SGCG1', 'SLC47A2', 'SOSTDC1', 'WIF1', 
            'ADGRD1', 'ANGTL1', 'CP', 'ENTPD1', 'EPHB6', 'ITIH5', 'LOXL1', 'MMP27', 'PCOLCE', 'PDPN', 'QPCT', 'SERPINF1', 'SFRP2', 'SVEP1', 'THSB2', 
            'THBS3', 'THBS4', 'XG']

res_A3 = retrieve_enrichr(human_A3, org='Human', name='A3')
df_GOs = pd.concat([df_GOs, res_A3.iloc[:N]])
res_A3.iloc[:N]

In [ ]:
human_A4 = ['ACE', 'AIF1L', 'APCDD1L', 'C1QTNF3', 'CD248', 'CD34', 'CILP', 'DBN1', 'EMILIN2', 'EMP3', 'FABP3', 'FAM180B', 'FBN1', 'HSD3B7', 
            'LINC01133', 'MATN4', 'PAMR1', 'PCOLCE2', 'PRG4', 'PTGIS', 'SEMA3C', 'SEMA3E', 'SH3BGRL3', 'TPPP3', 'TRAC', 'WNT10B', 'YWHAH', 
            'ABCC9', 'ACKR3', 'AGTR1', 'AQP1', 'CD151', 'CERCAM', 'CLEC3B', 'CLU', 'CRIP1', 'DKK1', 'DPP4', 'EBF3', 'FNDC1', 'GALNT15', 
            'GPX3', 'GSN', 'IGFBP6', 'NPR1', 'PIEZO2', 'PPIC', 'SCARA5', 'SEMA3B', 'SLC29A1', 'SLPI', 'THY1', 'TIMP2', 'TNXB', 
            'TPM1', 'WNT2', 'XG']

res_A4 = retrieve_enrichr(human_A4, org='Human', name='A4')
df_GOs = pd.concat([df_GOs, res_A4.iloc[:N]])
res_A4.iloc[:N]

In [ ]:
human_B1 = ['ADAMTS4', 'APLNR', 'BAZ1A', 'C11orf96', 'CDKN1A', 'CHI3L1', 'CXCL1', 'CXCL2', 'CXCL3', 'DYRK3', 'ELL2', 'ERRFI1', 'FOSL1', 
            'GCH1', 'GEM', 'HMGA1', 'IER3', 'IL6', 'KDM6B', 'KPNA2', 'MMP1', 'MMP3', 'NFIL3', 'NFKB1', 'NR4A3', 'PPP1R15B', 'REL', 'TIPARP', 
            'TNFAIP6', 'TNFSF14', 'ZC3H12A', 'ZNF331', 'ARID5B', 'AVPR1A', 'CEBPB', 'CEBPD', 'CHD1', 'DNAJA1', 'HLA-DRB1', 'ICAM1', 'IRF1', 
            'JUNB', 'MAFF', 'NFKBIA', 'PIM3', 'PLAUR', 'PNRC1', 'PPP1R15A', 'SOD2', 'TNFAIP3', 'TNFRSF10B', 'TSHZ2', 'UAP1', 'WTAP1',]

res_B1 = retrieve_enrichr(human_B1, org='Human', name='B1')
df_GOs = pd.concat([df_GOs, res_B1.iloc[:N]])
res_B1.iloc[:N]

In [ ]:
human_B2 = ['ADRA2A', 'APOC1', 'APOE', 'CLSTN3', 'CXCL12', 'CYP7B1', 'HLA-B', 'ICAM2', 'IGFBP7', 'IL33', 'IL34', 'JAK3', 'LPAR4', 'MSC', 
            'MUSK', 'PCDH17', 'PPDPFL', 'RBP5', 'SCN4B', 'SDK1', 'SLC9A3R2', 'SLCO2B1', 'ST8SIA1', 'TMEM150C', 'TNFSF13B', 'TYMP', 'VCAM1', 
            'ANKRD29', 'APLNR', 'C3', 'C6', 'C7', 'CCDC146', 'COX4I2', 'CYGB', 'EPHX1', 'FXYD6', 'GGT5', 'HMGCLL1', 'NFIB', 'NLGN4X', 
            'NOTCH3', 'RARRES2', 'SLIT2', 'TMEM176A', 'TMEM176B', 'TSHZ2']

res_B2 = retrieve_enrichr(human_B2, org='Human', name='B2')
df_GOs = pd.concat([df_GOs, res_B2.iloc[:N]])
res_B2.iloc[:N]

In [ ]:
human_B3 = ['ACHE', 'BIRC3', 'CCL19', 'CD74', 'CH25H', 'CSF1', 'CRB2', 'CTSH', 'CX3CL1', 'EGR1', 'EXOC3L4', 'HAS2', 'HLA-B', 'HLA-F', 'IL15', 
            'IL4I1', 'IRF8', 'OLFM2', 'POPDC2', 'PTPRT', 'RGS16', 'TYMP', 'UBD', 'ADRA2A', 'ANKRD29', 'CCDC146', 'CCL2', 'CD82', 'CLSTN3', 
            'CYP7B1', 'IL32', 'IL34', 'IRF1', 'JAK3', 'LPAR4', 'PIM3', 'RBP5', 'SDK1', 'SLC2A3', 'SOCS3', 'TMEM150C', 'TNC', 'TNFSF13B', 
            'VCAM1', 'ZFP36']

res_B3 = retrieve_enrichr(human_B3, org='Human', name='B3')
df_GOs = pd.concat([df_GOs, res_B3.iloc[:N]])
res_B3.iloc[:N]

In [ ]:
human_B4 = ['AADAC', 'ADA', 'ATP8B4', 'C6', 'CD36', 'EFEMP1', 'FGF10', 'FZD4', 'GDF10', 'GPX3', 'HHIP', 'HSPB6', 'IGF2', 'MGP', 'MYOC', 
            'PHLDA3', 'PLA2G2A', 'PLA2G5', 'PLBD1', 'PPARG', 'PPP1R1A', 'PQLC2L', 'SERPINA5', 'SIX1', 'SULT1A1', 'TNNT3', 'ZFHX4', 
            'ABCA10', 'APOC1', 'C7', 'CFH', 'EBF3', 'FMO1', 'GGT5', 'GSN', 'HMGCLL1', 'IGF1', 'ITM2A', 'LSP1', 'MGST1', 'NTRK2', 
            'PODN', 'RARRES2', 'STMN2', 'TXNIP', 'WNT11', ]

res_B4 = retrieve_enrichr(human_B4, org='Human', name='B4')
df_GOs = pd.concat([df_GOs, res_B4.iloc[:N]])
res_B4.iloc[:N]

In [ ]:
human_C1 = ['ACAN1', 'ACTA2', 'ADAMTS18', 'ANOs1', 'ARHGAP44', 'BCL11B', 'CALD1', 'CFHR1', 'CNN2', 
            'COL11A1', 'COL21A1', 'DOK6', 'DPEP1', 'EGFL6', 'EVA1A', 'F2RL2', 'GNAO1', 'HS6ST2', 
            'LGR4', 'LRRN3', 'MAGI1', 'MEF2C', 'MME', 'PAFAH1B3', 'PTCH1', 'RANBP3L', 'RBFOX1', 
            'RBP4', 'SLC26A7', 'TMEM171', 'UGT3A2', 'VIPR2', 'WDR86', 'WFDC1', 'ALX4', 'C1QTNF6', 
            'CPXM2', 'CYYR1', 'DKK3', 'F2R', 'HAPLN1', 'KIF26B', 'LAMC3', 'MFAP2', 'PCDH19', 'PDE1A', 
            'PPP1R14A', 'RFLNB', 'SGIP1', 'THSD4', 'TENM3', 'TRIL', 'TSPAN13']

res_C1 = retrieve_enrichr(human_C1, org='Human', name='C1')
df_GOs = pd.concat([df_GOs, res_C1.iloc[:N]])
res_C1.iloc[:N]

In [ ]:
human_C2 = ['ARHGAP15', 'ATP1B1', 'BTBD11', 'CADM2', 'CCDC68', 'CCK', 'CHADL', 'CHST15', 'COCH', 
            'COL24A1', 'CPNE5', 'CRABP1', 'EMB', 'FGF13', 'FIBIN', 'GAP43', 'GRP', 'HSPA2', 'KCNAB1', 
            'LRIG1', 'MAB21L2', 'MKX', 'NDNF', 'P3H2', 'PCSK9', 'PLPP5', 'PLXC1', 'TRIB2', 'TRPM3', 
            'VEGFD', 'WNT10A', 'ZC2HC1C', 'CADM1', 'F13A1', 'IGFBP5', 'MEOX2', 'NECAB1', 
            'NOTUM', 'NFATC2', 'PTGFR', 'PTH1R', 'PTPRK', 'TCF4', 'TNMD', 'TNN', 'TRPS1',]

res_C2 = retrieve_enrichr(human_C2, org='Human', name='C2')
df_GOs = pd.concat([df_GOs, res_C2.iloc[:N]])
res_C2.iloc[:N]

In [ ]:
human_C3 = ['ADAM12', 'BGN', 'C1QTNF6', 'EDIL3', 'HAPLN', 'KLK4', 'MMP11', 'P4HA3', 
            'POSTN', 'RAS11LB', 'RUNX2', 'SEMA5A', 'SGIP1', 'SLC5A3', 'TAC1', 'TENM3', 
            'TSPAN13', 'ACAN', 'ASPN', 'COL5A2', 'COL8A2', 'ELN', 'F2R', 'LOXL2', 'LRRC15', 
            'MDK', 'PMEPA1', 'SDC1', 'TRIL']

res_C3 = retrieve_enrichr(human_C3, org='Human', name='C3')
df_GOs = pd.concat([df_GOs, res_C3.iloc[:N]])
res_C3.iloc[:N]

In [ ]:
human_C5 = ['AOPEP', 'BMP7', 'CDH11', 'CENPW', 'CNTN4', 'CXCR4', 'FBXO32', 'GRIK1', 'IGFBP3', 
            'IQGAP2', 'KRT9', 'LMO3', 'LUZP2', 'MAGI1', 'MARCKSL1', 'PDE3A', 'PGM2L1', 'PKP4', 
            'PRDM1', 'PREX2', 'PRLR', 'SLC5A3', 'SOX18', 'TFAP2A', 'TFAP2C', 'TPD52', 'WNT5A', 
            'ARHGAP44', 'CRABP1', 'DCXR', 'DIO2', 'HEY2', 'HMGB3', 'HPSE2', 'INHBA', 'JAG1', 
            'KRT17', 'LMO4', 'MRPS6', 'NKD1', 'PAFAH1B3', 'ROBO1', 'ROBO2', 'RSPO3', 'SEMA3G', 
            'SNAI2', 'SPON2', 'TBX3', 'TRPS1']

res_C5 = retrieve_enrichr(human_C5, org='Human', name='C5')
df_GOs = pd.concat([df_GOs, res_C5.iloc[:N]])
res_C5.iloc[:N]

In [ ]:
human_D1 = ['AFAP1L2', 'ALDH1A2', 'ANGPTL7', 'ATP1A2', 'BAMBI', 'C2orf40', 'ECRG4', 'CDH19', 
            'CHN1', 'COL28A1', 'COL9A3', 'CRISPLD1', 'CYP1B1', 'DUSP5', 'EGR3', 'ENTPD2', 'ETV1', 
            'ETV4', 'FMO2', 'FMO3', 'FOXC2', 'LZTS1', 'MCTP1', 'MRAS', 'MYBPC1', 'PODNL1', 
            'PSAT1', 'SCRG1', 'SOX8', 'SOX9', 'TIAM1', 'CD200', 'CLDN1', 'COL8A1', 'EBF2', 
            'EFNA1', 'EFNB1', 'FGFBP2', 'FOXS2', 'KLK1', 'LDLR', 'LPCAT2', 'MARCKSL1', 'MTUS1', 
            'NRP2', 'P2RY14', 'PLK2', 'PTGDS', 'SCN7A', 'SFRP4', 'SPRY2', 'TENM2', 'TM4SF1']

res_D1 = retrieve_enrichr(human_D1, org='Human', name='D1')
df_GOs = pd.concat([df_GOs, res_D1.iloc[:N]])
res_D1.iloc[:N]

In [ ]:
human_D2 = ['ADAMTSL3', 'ADAMTSL4', 'ADAMTSL5', 'AQP3', 'BNC2', 'C19orf33', 'CAV2', 'CAVIN2', 
            'CCL13', 'CTTNBP2', 'DACT1', 'DOCK9', 'EGFLAM', 'GFRA2', 'GPC6', 'HRH1', 'IGF2', 
            'INMT', 'ISYNA1', 'ITGA6', 'ITGB4', 'KLF5', 'KRT19', 'NDUFA4L2', 'NGFR', 'PALMD', 
            'PDZRN4', 'PEAR1', 'SBSPON', 'SDPR', 'CAVIN2', 'SLC22A3', 'SHISA3', 'SLC2A1', 
            'TNNC1', 'ECRG4', 'C2orf40', 'CSRP1', 'DAAM1', 'DDIT4L', 'DUSP5', 'EBF2', 'GAB1', 
            'LMO7', 'MATN2', 'NDRG2', 'PLEKHA4', 'S100B', 'SFRP4', 'SORBS1', 'TAGLN', 'TJP1', 
            'TM4SF1', 'TXNIP', 'UACA', 'VIT']
res_D2 = retrieve_enrichr(human_D2, org='Human', name='D2')
df_GOs = pd.concat([df_GOs, res_D2.iloc[:N]])
res_D2.iloc[:N]

In [ ]:
human_E1 = ['CMKLR1', 'COL26A1', 'EGR2', 'FHAD1', 'FZD7', 'GMFG', 'HOPX', 'IGFBP2', 'LFNG', 
            'MAP2', 'NECAB1', 'NTRK3', 'PLA2G5', 'PLEKHA6', 'PRDM8', 'RAMP1', 'SAMD5', 'SIX1', 
            'SLC22A3', 'THSD1', 'A2M', 'ANTXR2', 'CPE', 'IGF1', 'ITM2A', 'MEOX1', 'MGP', 'MMP16', 
            'OLFML2A', 'PDGFD', 'PTN', 'RGCC', 'RGMA', 'SFRP1', 'SPON2', 'SPRY1', 'SPRY2', 
            'STMN2', 'TCF7L2', 'TIMP3', 'TMPO', 'TNFRSF21', 'UNC5B', 'WDR86']

res_E1 = retrieve_enrichr(human_E1, org='Human', name='E1')
df_GOs = pd.concat([df_GOs, res_E1.iloc[:N]])
res_E1.iloc[:N]

In [ ]:
df_GOs.to_csv('results/GOEA.csv', index=None)

In [ ]:
df_GOs_mod = df_GOs.copy()

In [ ]:
df_GOs_mod = df_GOs_mod[['Population', 'Term', 'Adjusted P-value', 'Genes']]

In [ ]:
df_GOs_mod.to_csv('results/GOEA_mod.csv', index=None)

In [ ]:
os.system('jupyter nbconvert --to html 9HM_GO_main_populations.ipynb')